In [ ]:
import random
from enum import Enum

In [ ]:
# --- Constants and Enumerations Definitions ---

# Fixed System Parameters (Example values)
L = 200        # Road Length (cells)
V_MAX = 5      # Maximum Speed (cells/time step)
T_GREEN = 40   # Green Light Period (time steps)
T_CYCLE = 2 * T_GREEN # Full Cycle (assuming symmetry)
INJECTION_RATE = 0.1 # Probability of injection 'a'

# Base Parameters (Example values)
P_B = 0.1      # Random Braking Probability (NaSch)
P_CHG = 0.8    # Lane Change Probability
P_RED = 0.1    # Red Light Violation Probability
P_SKID = 0.05  # Braking Failure Probability (Rear-end collision)

class Road(Enum):
    """Represents the two perpendicular roads of the intersection."""
    R1 = 1 # Vertical Axis (North-South)
    R2 = 2 # Horizontal Axis (West-East)
    
class Lane(Enum):
    """Represents the two lanes of a road."""
    LEFT = 0
    RIGHT = 1

class TrafficLightState(Enum):
    """Traffic light state for a road (R1 or R2)."""
    RED = 0
    GREEN = 1

class Weather(Enum):
    """Weather condition for the simulation."""
    NORMAL = 0
    RAINY = 1 # "Rainy day" state (global modifier)

# ----------------------------------------------------
# Vehicle Class
# ----------------------------------------------------

class Vehicle:
    """Represents a single vehicle in the model."""
    def __init__(self, vehicle_id, road: Road, lane: Lane, vmax, p_red, p_skid):
        self.id = vehicle_id
        self.road = road # R1 or R2
        self.lane = lane # LEFT or RIGHT
        self.position = 0
        self.velocity = 0
        self.v_max = vmax
        
        # Individual parameters (can be modified by weather)
        self.p_red = p_red
        self.p_skid = p_skid
        
        # Collision status
        self.collided = False

# ----------------------------------------------------
# Intersection Model Class
# ----------------------------------------------------

class IntersectionModel:
    """
    Main simulation managing roads, vehicles, traffic light, and accidents.
    It's an extension of the Nagel-Schreckenberg model.
    """
    def __init__(self, length, vmax, t_green, injection_rate, p_b, p_chg, p_red, p_skid):
        
        # Simulation Parameters
        self.L = length
        self.V_MAX_BASE = vmax # Base max velocity for new vehicles
        self.T_GREEN = t_green
        self.INJECTION_RATE = injection_rate
        self.P_B_BASE = p_b
        self.P_CHG_BASE = p_chg
        
        # Simulation objects collections
        # The 'grid' is a dictionary-based representation of the road.
        # {Road.R1: {Lane.LEFT: [None] * L, Lane.RIGHT: [None] * L}, ...}
        self.grid = {
            r: {l: [None] * self.L for l in Lane}
            for r in Road
        }
        self.vehicles = [] # List of all active Vehicle objects
        self.next_vehicle_id = 0
        self.time_step = 0
        
        # Traffic Light
        self.traffic_light = {Road.R1: TrafficLightState.GREEN, Road.R2: TrafficLightState.RED}
        
        # Base Risk Parameters
        self.P_RED_BASE = p_red
        self.P_SKID_BASE = p_skid
        
        # Output Variables (Metrics to Measure)
        self.N_lateral = 0  # Total count of lateral collisions (red light violation)
        self.N_rear_end = 0  # Total count of rear-end collisions (braking failure)
        self.N_vehicles = 0 # Total number of vehicles that have circulated
        self.throughput = 0 # Number of vehicles crossing the intersection per unit time
        
        # Environmental state and modifiers
        self.weather = Weather.NORMAL
        self.P_B = self.P_B_BASE
        self.P_CHG = self.P_CHG_BASE
        self.P_RED = self.P_RED_BASE
        self.P_SKID = self.P_SKID_BASE
        
        # Define the intersection cell (the cell right at the crossing)
        self.intersection_cell = self.L // 2

    def set_weather(self, weather: Weather):
        """Applies the modifiers for the weather condition (rainy day)."""
        self.weather = weather
        if weather == Weather.RAINY:
            # Modifications due to rain
            self.P_RED = self.P_RED_BASE * 0.5  # P_red is reduced (more cautious)
            self.P_CHG = self.P_CHG_BASE * 0.5  # P_chg is reduced (discourages lane changes)
            self.P_SKID = self.P_SKID_BASE * 2.0 # P_skid increases significantly (braking failure/aquaplaning)
            self.P_B = self.P_B_BASE * 1.5      # P_b increases (longer braking distance/more cautious braking)
        else:
            # Revert to base values
            self.P_RED = self.P_RED_BASE
            self.P_CHG = self.P_CHG_BASE
            self.P_SKID = self.P_SKID_BASE
            self.P_B = self.P_B_BASE

        # Update risk parameters in all existing vehicles
        for vehicle in self.vehicles:
            vehicle.p_red = self.P_RED
            vehicle.p_skid = self.P_SKID
            
    def update_traffic_light(self):
        """Updates the traffic light state every T_GREEN time steps."""
        # Full cycle T = 2 * T_GREEN
        if (self.time_step % self.T_GREEN) == 0:
            # Switch lights
            if self.traffic_light[Road.R1] == TrafficLightState.GREEN:
                self.traffic_light[Road.R1] = TrafficLightState.RED
                self.traffic_light[Road.R2] = TrafficLightState.GREEN
            else:
                self.traffic_light[Road.R1] = TrafficLightState.GREEN
                self.traffic_light[Road.R2] = TrafficLightState.RED

    def inject_vehicle(self):
        """Tries to inject a new vehicle into the system."""
        # Implementation of vehicle injection with INJECTION_RATE 'a'
        # Should choose a road (R1/R2) and a lane (LEFT/RIGHT) if the starting cell (position 0) is free.
        pass # Injection logic (to be implemented)
        
    def find_front_vehicle(self, vehicle: Vehicle) -> Vehicle | None:
        """Finds the vehicle object directly ahead in the same lane."""
        road = self.grid[vehicle.road][vehicle.lane]
        # Search from the cell in front of the vehicle to the end
        for i in range(vehicle.position + 1, self.L):
            if road[i] is not None:
                return road[i]
        return None # No vehicle found

    def find_front_gap(self, vehicle: Vehicle) -> int:
        """
        Calculates the distance (gap) to the vehicle ahead.
        This is the number of empty cells *between* this car and the next one.
        """
        road = self.grid[vehicle.road][vehicle.lane]
        # Iterate from the cell in front of the vehicle to the end
        for i in range(vehicle.position + 1, self.L):
            if road[i] is not None:
                # Found a vehicle, gap is distance to it
                return i - vehicle.position - 1
        # No vehicle found, gap is to the end of the road
        return self.L - vehicle.position - 1

    def apply_nash_rules(self):
        """
        Applies the NaSch update rules and collision logic in three phases:
        1. Calculation: Determine intended moves.
        2. Collision Check: Detect and resolve lateral and rear-end collisions.
        3. Update: Apply final moves to the grid.
        """
        
        # Stores the intended state {'pos': int, 'vel': int} for each vehicle
        new_vehicles_state = {}
        # Stores vehicles intending to enter the intersection {Road.R1: [v1, v2], R2: [v3]}
        intersection_entrants = {Road.R1: [], Road.R2: []}

        # --- PHASE 1: Calculate intended moves for all vehicles ---
        
        for vehicle in self.vehicles:
            if vehicle.collided:
                # If already collided, it doesn't move
                new_vehicles_state[vehicle] = {'pos': vehicle.position, 'vel': 0}
                continue

            v_i = vehicle.velocity
            # d_i is the gap to the *next vehicle*
            d_i = self.find_front_gap(vehicle)
            
            # --- NaSch Rules 1 (Acceleration) ---
            v_new = min(v_i + 1, vehicle.v_max)
            
            # --- NaSch Rules 2 (Deceleration - Car Ahead) ---
            # Slow down to avoid hitting the car in front
            v_new = min(v_new, d_i)
            
            # --- Intersection Logic (Part of Deceleration) ---
            # g_i is the gap to the *intersection stop line*
            gap_to_intersection = self.intersection_cell - vehicle.position - 1
            
            # Check if vehicle is approaching a RED light
            if self.traffic_light[vehicle.road] == TrafficLightState.RED:
                # If the car is close enough to cross on its next move...
                # (v_new > gap_to_intersection) means its intended speed will cross the line
                if v_new > gap_to_intersection:
                    # **NEW LOGIC: Decide whether to violate the red light**
                    if random.random() < vehicle.p_red:
                        # **INTENT TO VIOLATE RED LIGHT**
                        # The vehicle *ignores* the red light.
                        # v_new remains as calculated (limited only by car ahead)
                        pass 
                    else:
                        # **INTENT TO STOP**
                        # Decelerate to stop *before* the intersection
                        v_new = max(0, gap_to_intersection) # Stop at the line
            
            # --- NaSch Rules 3 (Randomization) ---
            if v_new > 0 and random.random() < self.P_B: # P_B is modified by rain
                v_new -= 1
            
            # --- Braking Failure Check (Rear-End Collision) ---
            # If the car's *final* velocity (after rules) is *still* greater than
            # the gap to the car in front, a skid (braking failure) might occur.
            if v_new > d_i and random.random() < vehicle.p_skid:
                # Collision occurs!
                self.N_rear_end += 1
                vehicle.collided = True
                front_vehicle = self.find_front_vehicle(vehicle)
                if front_vehicle:
                    front_vehicle.collided = True # The car in front is also hit
                
                # Stop at the collision point (cell behind the front car)
                v_new = 0
                new_pos = vehicle.position + d_i
            else:
                # --- Car Motion (Intended) ---
                # This is the normal, collision-free movement
                v_new = min(v_new, d_i) # Final safety check
                new_pos = vehicle.position + v_new

            # Store the intended move
            new_vehicles_state[vehicle] = {'pos': new_pos, 'vel': v_new}
            
            # --- Check if this move enters the intersection ---
            # A vehicle enters if its old pos was *before* and new pos is *at or after*
            if vehicle.position < self.intersection_cell <= new_pos and not vehicle.collided:
                intersection_entrants[vehicle.road].append(vehicle)

        # --- PHASE 2: Check for Lateral Collisions ---
        
        # **NEW LOGIC: If *both* roads have vehicles entering the intersection...**
        if intersection_entrants[Road.R1] and intersection_entrants[Road.R2]:
            self.N_lateral += 1
            
            # Mark ALL vehicles entering the intersection as collided
            # This is a simplification; a more complex model would check lane-by-lane
            for road_entrants in intersection_entrants.values():
                for vehicle in road_entrants:
                    if not vehicle.collided: # Don't re-collide a rear-ended car
                        vehicle.collided = True
                        # The vehicle stops *at* the collision point (the intersection cell)
                        new_vehicles_state[vehicle]['pos'] = self.intersection_cell
                        new_vehicles_state[vehicle]['vel'] = 0

        # --- PHASE 3: Apply Final State and Update Grid ---
        
        # Clear the grid for the new time step
        self.grid = {r: {l: [None] * self.L for l in Lane} for r in Road}
        
        vehicles_to_remove = []

        for vehicle in self.vehicles:
            # Get the final state (which may have been modified by collision)
            final_pos = new_vehicles_state[vehicle]['pos']
            final_vel = new_vehicles_state[vehicle]['vel']

            if vehicle.collided:
                vehicle.velocity = 0 # Ensure velocity is 0
                # If the collided vehicle is still on the board, place it
                if final_pos < self.L:
                    vehicle.position = final_pos
                    # Place it in the grid so it blocks traffic
                    self.grid[vehicle.road][vehicle.lane][final_pos] = vehicle
                else:
                    # Collided and also tried to leave? Just remove it.
                    vehicles_to_remove.append(vehicle)
            
            elif final_pos >= self.L:
                # Vehicle leaves the system
                vehicles_to_remove.append(vehicle)
                self.throughput += 1
            
            else:
                # Vehicle moves to new cell
                vehicle.position = final_pos
                vehicle.velocity = final_vel
                self.grid[vehicle.road][vehicle.lane][final_pos] = vehicle

        # Remove vehicles that left or collided off-map
        for vehicle in vehicles_to_remove:
            self.vehicles.remove(vehicle)

        self.time_step += 1
        
    def run_simulation(self, steps, rainy=False):
        """Runs the simulation for a number of steps."""
        self.set_weather(Weather.RAINY if rainy else Weather.NORMAL)
        
        print(f"--- Running simulation for {steps} steps (Weather: {self.weather.name}) ---")
        for i in range(steps):
            self.update_traffic_light()
            self.inject_vehicle()
            self.apply_nash_rules()
            # In a real run, you'd collect data here
        print("--- Simulation complete ---")

    def get_metrics(self):
        """Returns the output metrics of the simulation."""
        return {
            'N_lateral': self.N_lateral,
            'N_rear_end': self.N_rear_end,
            'N_vehicles': self.N_vehicles,
            'Throughput': self.throughput,
            'Accident_Ratio_Lateral_to_RearEnd': self.N_lateral / (self.N_rear_end + 1e-6) # Avoid div by zero
        }

In [ ]:
# ----------------------------------------------------
# Execution (Example of usage)
# ----------------------------------------------------

if __name__ == "__main__":
    
    # Base parameters for initialization
    L_INIT = 200
    V_MAX_INIT = 5
    T_GREEN_INIT = 40
    INJECTION_RATE_INIT = 0.1
    P_B_INIT = 0.1
    P_CHG_INIT = 0.8
    P_RED_INIT = 0.1
    P_SKID_INIT = 0.05
    
    # Simulation under NORMAL conditions
    model_normal = IntersectionModel(L_INIT, V_MAX_INIT, T_GREEN_INIT, INJECTION_RATE_INIT, P_B_INIT, P_CHG_INIT, P_RED_INIT, P_SKID_INIT)
    # model_normal.run_simulation(steps=1000)
    # metrics_normal = model_normal.get_metrics()
    
    # Simulation under RAINY conditions
    model_rainy = IntersectionModel(L_INIT, V_MAX_INIT, T_GREEN_INIT, INJECTION_RATE_INIT, P_B_INIT, P_CHG_INIT, P_RED_INIT, P_SKID_INIT)
    # model_rainy.run_simulation(steps=1000, rainy=True)
    # metrics_rainy = model_rainy.get_metrics()
    
    # Print parameter comparison
    print(f"Base Parameters: P_RED={P_RED_INIT}, P_SKID={P_SKID_INIT}, P_CHG={P_CHG_INIT}, P_B={P_B_INIT}")
    
    # Print parameters modified by Rain
    model_rainy.set_weather(Weather.RAINY)
    print(f"\nRAINY Parameters (Modifiers Applied): P_RED={model_rainy.P_RED}, P_SKID={model_rainy.P_SKID}, P_CHG={model_rainy.P_CHG}, P_B={model_rainy.P_B}")
    # print("\nResults (Normal):", metrics_normal)
    # print